In [1]:
import pandas as pd
from sqlalchemy import create_engine

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

In [28]:
year = 2022
month = 1
year, month

(2022, 1)

### Calculate profits

In [11]:
format_dict = {
    'qty':'{:,}',
    'buy_price':'{:.2f}','sell_price':'{:.2f}',
    'profit':'{:,.2f}','buy_amt':'{:,.2f}','sell_amt':'{:,.2f}',
    'number':'{:,}',
    'amt':'{:,.2f}','net':'{:,.2f}',
    'ppu':'{:.4f}',
    'x_date':'{:%Y-%m-%d}',    
    'p_date':'{:%Y-%m-%d}',  
}

In [4]:
sql = '''
SELECT name, sells.date AS sell_date, qty,
buys.price AS buy_price, buys.net AS buy_amt, 
sells.price AS sell_price, sells.net AS sell_amt, profit 
FROM sells 
JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = %s AND MONTH(sells.date) = %s
ORDER BY sells.date, name'''
sql = sql % (year, month)
print(sql)


SELECT name, sells.date AS sell_date, qty,
buys.price AS buy_price, buys.net AS buy_amt, 
sells.price AS sell_price, sells.net AS sell_amt, profit 
FROM sells 
JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = 2022 AND MONTH(sells.date) = 5
ORDER BY sells.date, name


In [5]:
sells_df = pd.read_sql(sql, conpf)
sells_df.style.format(format_dict)

,name,sell_date,qty,buy_price,buy_amt,sell_price,sell_amt,profit
0,SPALI,2022-05-03,"3,000",22.00,"66,146.18",22.10,"66,153.15",6.97
1,PTT,2022-05-06,"3,000",38.50,"115,755.83",37.50,"112,250.82","-3,505.01"
2,DOHOME,2022-05-10,"2,400",20.25,"48,707.64",19.40,"46,456.87","-2,250.77"
3,SPALI,2022-05-11,"6,000",22.00,"132,292.37",21.20,"126,918.27","-5,374.10"
4,RATCH,2022-05-12,"2,000",45.75,"91,702.67",39.75,"79,323.91","-12,378.76"
5,RATCH,2022-05-13,"2,000",45.75,"91,702.67",39.00,"77,827.24","-13,875.43"
6,PTT,2022-05-17,"3,000",38.50,"115,755.83",37.00,"110,754.15","-5,001.68"
7,RATCH,2022-05-17,"2,000",45.75,"91,702.67",38.50,"76,829.45","-14,873.22"
8,HREIT,2022-05-23,100,8.95,896.98,8.40,838.14,-58.84
9,RJH,2022-05-23,"4,500",33.00,"148,828.92",34.00,"152,661.12","3,832.20"


In [6]:
flt_format = '{:,.2f}'
flt_format.format(sells_df.profit.sum())

'-45,977.40'

### Dividend

In [14]:
pd.read_sql_query('SELECT * FROM dividends ORDER BY id DESC LIMIT 1', conpf).style.format(format_dict)

,id,stock_id,name,year,quarter,number,ppu,amt,net,x_date,p_date
0,329,166,DCC,2022,1,"60,000",0.0550,"3,300.00","2,970.00",2022-05-13,2022-05-31


In [29]:
sql = """
SELECT name, p_date, number,
ppu, amt, net 
FROM dividends
WHERE YEAR(p_date) = %s AND MONTH(p_date) = %s
ORDER BY p_date, name"""
sql = sql % (year, month)
print(sql)


SELECT name, p_date, number,
ppu, amt, net 
FROM dividends
WHERE YEAR(p_date) = 2022 AND MONTH(p_date) = 1
ORDER BY p_date, name


In [30]:
div_df = pd.read_sql(sql, conpf)
div_df.style.format(format_dict)

,name,p_date,number,ppu,amt,net


In [31]:
flt_format = '{:,.2f}'
flt_format.format(div_df.net.sum())

'0.00'